# 02. Funnel 분석 실습

SQL로 직접 퍼널 분석을 수행해봅니다.

**규칙:**
- 힌트는 최대한 안 보고 먼저 시도하세요
- 막히면 힌트를 펼쳐보세요
- 완료 후 `solution.sql`과 비교하세요

## 환경 설정

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path

# 데이터베이스 연결
DB_PATH = Path("../data/crm.db")
conn = sqlite3.connect(DB_PATH)

# SQL 실행 헬퍼 함수
def sql(query):
    """SQL 쿼리 실행 및 결과 반환"""
    return pd.read_sql(query, conn)

print("데이터베이스 연결 완료!")

---
## 준비: 이벤트 데이터 파악

분석 전에 항상 데이터를 먼저 확인하세요.

In [ ]:
# events 테이블 확인
sql("SELECT * FROM events LIMIT 10")

In [ ]:
# 이벤트 유형별 건수
sql("""
SELECT 
    event_type,
    COUNT(*) as event_count
FROM events
GROUP BY event_type
ORDER BY event_count DESC
""")

In [ ]:
# 데이터 기간 확인
sql("""
SELECT 
    MIN(event_date) as start_date,
    MAX(event_date) as end_date,
    COUNT(DISTINCT user_id) as unique_users,
    COUNT(DISTINCT session_id) as unique_sessions
FROM events
""")

---
# Mission 1: 기본 퍼널 구축

각 단계별 사용자 수를 계산해봅시다.

## Mission 1-1: 단계별 고유 사용자 수

**문제:** 각 이벤트 단계별로 고유 사용자 수를 계산하세요.

출력 컬럼: `event_type`, `unique_users`

In [ ]:
# TODO: 단계별 고유 사용자 수를 계산하는 SQL을 작성하세요
sql("""
SELECT 
    event_type,
    _____ as unique_users
FROM events
GROUP BY _____
ORDER BY unique_users DESC
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- 고유 사용자 수: `COUNT(DISTINCT user_id)`
- GROUP BY는 `event_type`으로

</details>

## Mission 1-2: 퍼널 순서대로 정렬

**문제:** 퍼널 단계 순서대로 (page_view → product_view → add_to_cart → checkout_start → purchase) 정렬하세요.

**힌트:** CASE WHEN을 사용하여 순서를 지정할 수 있습니다.

In [ ]:
# TODO: 퍼널 순서대로 정렬된 결과를 출력하세요
sql("""
SELECT 
    event_type,
    COUNT(DISTINCT user_id) as unique_users,
    CASE event_type
        WHEN 'page_view' THEN _____
        WHEN 'product_view' THEN _____
        WHEN 'add_to_cart' THEN _____
        WHEN 'checkout_start' THEN _____
        WHEN 'purchase' THEN _____
    END as step_order
FROM events
GROUP BY event_type
ORDER BY step_order
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- 순서: page_view=1, product_view=2, add_to_cart=3, checkout_start=4, purchase=5

</details>

---
# Mission 2: 전환율 계산

각 단계 간 전환율을 계산해봅시다.

## Mission 2-1: 전체 퍼널 전환율

**문제:** 첫 단계(page_view) 대비 각 단계의 전환율을 계산하세요.

```
전환율 = (해당 단계 사용자 수 / 첫 단계 사용자 수) × 100
```

출력 컬럼: `step_order`, `event_type`, `unique_users`, `conversion_rate`

In [ ]:
# TODO: 첫 단계 대비 전환율을 계산하는 SQL을 작성하세요
sql("""
WITH funnel AS (
    SELECT 
        event_type,
        COUNT(DISTINCT user_id) as unique_users,
        CASE event_type
            WHEN 'page_view' THEN 1
            WHEN 'product_view' THEN 2
            WHEN 'add_to_cart' THEN 3
            WHEN 'checkout_start' THEN 4
            WHEN 'purchase' THEN 5
        END as step_order
    FROM events
    GROUP BY event_type
),
first_step AS (
    SELECT unique_users as total_users
    FROM funnel
    WHERE event_type = 'page_view'
)
SELECT 
    f.step_order,
    f.event_type,
    f.unique_users,
    ROUND(_____ * 100, 2) as conversion_rate
FROM funnel f, first_step fs
ORDER BY f.step_order
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- 전환율: `f.unique_users * 1.0 / fs.total_users`
- `* 1.0`을 곱해야 정수 나눗셈을 피할 수 있습니다

</details>

## Mission 2-2: 단계별 전환율 (Step-to-Step)

**문제:** 이전 단계 대비 다음 단계로의 전환율을 계산하세요.

```
단계별 전환율 = (현재 단계 사용자 수 / 이전 단계 사용자 수) × 100
```

**비즈니스 맥락:** 어느 단계에서 가장 많이 이탈하는지 파악 (병목 지점 발견)

In [ ]:
# TODO: 단계별 전환율을 계산하는 SQL을 작성하세요
sql("""
WITH funnel AS (
    SELECT 
        event_type,
        COUNT(DISTINCT user_id) as unique_users,
        CASE event_type
            WHEN 'page_view' THEN 1
            WHEN 'product_view' THEN 2
            WHEN 'add_to_cart' THEN 3
            WHEN 'checkout_start' THEN 4
            WHEN 'purchase' THEN 5
        END as step_order
    FROM events
    GROUP BY event_type
)
SELECT 
    curr.step_order,
    curr.event_type,
    curr.unique_users,
    prev.unique_users as prev_step_users,
    ROUND(_____ * 100, 2) as step_conversion_rate
FROM funnel curr
LEFT JOIN funnel prev ON curr.step_order = prev.step_order + 1
ORDER BY curr.step_order
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- 전환율: `curr.unique_users * 1.0 / prev.unique_users`
- 첫 단계는 이전 단계가 없으므로 NULL이 됩니다
- `COALESCE`로 NULL 처리하거나, 첫 단계는 100%로 표시할 수 있습니다

</details>

**생각해보기:**
- 가장 전환율이 낮은 단계는 어디인가요? (병목 지점)
- 그 단계에서 사용자가 왜 이탈할까요?
- 어떻게 개선할 수 있을까요?

---
# Mission 3: 세그먼트별 퍼널 분석

디바이스, 채널 등 세그먼트별로 퍼널 성과를 비교해봅시다.

## Mission 3-1: 디바이스별 퍼널

**문제:** 디바이스별로 퍼널 전환율을 계산하세요.

**비즈니스 맥락:** 모바일 vs 데스크톱 어디에서 전환율이 더 좋은지 파악

In [ ]:
# TODO: 디바이스별 퍼널을 계산하는 SQL을 작성하세요
sql("""
WITH device_funnel AS (
    SELECT 
        device,
        event_type,
        COUNT(DISTINCT user_id) as unique_users
    FROM events
    GROUP BY device, event_type
),
device_first_step AS (
    SELECT device, unique_users as total_users
    FROM device_funnel
    WHERE event_type = 'page_view'
)
SELECT 
    f.device,
    f.event_type,
    f.unique_users,
    ROUND(_____ * 100, 2) as conversion_rate
FROM device_funnel f
JOIN device_first_step fs ON f.device = fs.device
ORDER BY f.device, 
    CASE f.event_type
        WHEN 'page_view' THEN 1
        WHEN 'product_view' THEN 2
        WHEN 'add_to_cart' THEN 3
        WHEN 'checkout_start' THEN 4
        WHEN 'purchase' THEN 5
    END
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- 디바이스별 첫 단계 사용자를 구해서 JOIN
- 전환율: `f.unique_users * 1.0 / fs.total_users`

</details>

## Mission 3-2: 채널별 최종 전환율 비교

**문제:** 유입 채널별로 최종 전환율(page_view → purchase)을 비교하세요.

출력 컬럼: `channel`, `page_views`, `purchases`, `overall_conversion_rate`

In [ ]:
# TODO: 채널별 최종 전환율을 계산하는 SQL을 작성하세요
sql("""
SELECT 
    channel,
    SUM(CASE WHEN event_type = 'page_view' THEN 1 ELSE 0 END) as page_views,
    SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) as purchases,
    ROUND(
        _____,
        2
    ) as overall_conversion_rate
FROM events
GROUP BY channel
ORDER BY overall_conversion_rate DESC
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- 조건부 집계: `SUM(CASE WHEN ... THEN 1 ELSE 0 END)`
- 전환율: `purchases * 100.0 / NULLIF(page_views, 0)`
- `NULLIF(x, 0)`은 x가 0이면 NULL 반환 (0으로 나누기 방지)

</details>

---
# Mission 4: 퍼널 드롭오프 분석

각 단계에서 얼마나 많은 사용자가 이탈하는지 분석해봅시다.

## Mission 4-1: 드롭오프 분석

**문제:** 각 단계에서의 이탈 수와 이탈율을 계산하세요.

```
이탈 수 = 이전 단계 사용자 - 현재 단계 사용자
이탈율 = (이탈 수 / 이전 단계 사용자) × 100
```

출력 컬럼: `step`, `event_type`, `users`, `dropoff_count`, `dropoff_rate`

In [ ]:
# TODO: 드롭오프 분석 SQL을 작성하세요
sql("""
WITH funnel AS (
    SELECT 
        event_type,
        COUNT(DISTINCT user_id) as users,
        CASE event_type
            WHEN 'page_view' THEN 1
            WHEN 'product_view' THEN 2
            WHEN 'add_to_cart' THEN 3
            WHEN 'checkout_start' THEN 4
            WHEN 'purchase' THEN 5
        END as step
    FROM events
    GROUP BY event_type
)
SELECT 
    curr.step,
    curr.event_type,
    curr.users,
    COALESCE(prev.users - curr.users, 0) as dropoff_count,
    ROUND(
        CASE 
            WHEN prev.users IS NULL THEN 0
            ELSE _____
        END,
        2
    ) as dropoff_rate
FROM funnel curr
LEFT JOIN funnel prev ON curr.step = prev.step + 1
ORDER BY curr.step
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- 이탈율: `(prev.users - curr.users) * 100.0 / prev.users`
- 첫 단계(page_view)는 이전 단계가 없으므로 이탈율 0

</details>

**생각해보기:**
- 이탈율이 가장 높은 단계는 어디인가요?
- 이 단계의 UX를 어떻게 개선할 수 있을까요?

---
# Mission 5: 실무 시나리오

**시나리오:**

> 프로덕트팀에서 전환율 개선 프로젝트를 진행하려고 합니다.
> 퍼널 데이터를 분석하여 다음을 도출하세요:
>
> 1. 가장 큰 병목 지점 (전환율이 가장 낮은 단계)
> 2. 디바이스별로 병목 지점이 다른지 확인
> 3. 개선 우선순위 권고안

**당신의 분석을 아래에 작성하세요:**

In [ ]:
# 자유롭게 분석 쿼리를 작성하세요

# 1. 전체 퍼널 병목 분석
sql("""
-- 여기에 쿼리 작성

""")

In [ ]:
# 2. 디바이스별 병목 비교
sql("""
-- 여기에 쿼리 작성

""")

### 나의 분석 결과 및 권고안

(여기에 분석 결과를 작성하세요)

1. **가장 큰 병목 지점**: 
2. **디바이스별 차이**: 
3. **개선 우선순위 권고안**: 

---
# 회고

실습을 마치며 아래 질문에 답해보세요.

1. **어떤 부분이 어려웠나요?**

2. **새롭게 배운 SQL 기법은?**

3. **실무에서 퍼널 분석을 활용한다면?**

4. **면접에서 이 내용을 어떻게 설명하겠습니까?**

In [ ]:
# 데이터베이스 연결 종료
conn.close()
print("실습 완료!")